In [21]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
base_dir = "C://DATA//train//49_kaggle//KaggleFun//HealthAnalytics//DATA"
df_train = pd.read_csv(os.path.join(base_dir,"pro_train.csv"))
df_test = pd.read_csv(os.path.join(base_dir,"pro_test.csv"))
df_train.head(5)

,Unnamed: 0,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,231676,0,7,1,4,2,1,2,2.0,8.0,0,1,4,4,6247.0,2
1,166821,0,7,1,2,3,3,2,1.0,2.0,0,1,6,6,8000.0,10
2,70566,1,2,1,2,3,2,3,3.0,7.0,0,2,3,2,4987.0,1
3,197982,1,2,1,2,2,1,3,3.0,14.0,1,1,3,6,7210.0,6
4,280389,3,13,1,4,3,2,1,2.0,8.0,2,1,2,4,3178.0,5


In [22]:
df_y = df_train.Stay
df_train = df_train.drop("Stay", axis = 1)
df_test_y = df_test.Stay
df_test = df_test.drop("Stay", axis=1)

In [23]:
print(df_train.shape)

(254750, 15)


# Currently, we have 15 features, we should try to narrow it down to 7 (half). Some features are deemed unimportant in A2_GS_xgboost, we can remove some of those as well

In [24]:
df_train = np.delete(np.array(df_train), [5,13], axis = 1)
print(df_train.shape)

(254750, 13)


In [29]:
df_test = np.delete(np.array(df_test), [5,13], axis=1)
print(df_test.shape)

(63688, 13)


In [26]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=7)
pca.fit(df_train)
X_train = pca.transform(df_train)
print(X_train.shape)

(254750, 7)


In [30]:
X_test = pca.transform(df_test)
print(X_test.shape)

(63688, 7)


In [35]:
def eva(model):
    y0 = model.predict(X_train)
    correct = 0
    df_y0 = df_y
    for i in range(len(y0)):
        if y0[i] == df_y0[i]: 
            correct += 1
    a1 = correct / len(X_train)
    
    y0 = model.predict(X_test)
    correct = 0
    df_y0 = df_test_y
    for i in range(len(y0)):
        if y0[i] == df_y0[i]: 
            correct += 1
    a2 = correct / len(df_test)
    
    print(a1,a2)
    return a1, a2

In [38]:
from xgboost import XGBClassifier

xgb_model=XGBClassifier(objective="multi:softmax",learning_rate=0.3,min_child_weight=1, max_depth=8, n_estimators=250)
xgb_model.fit(X_train,df_y)
print("Train Complete, Evaluating...")
# print("LR: {}, Depth: {}, Estimators: {}".format(learning_rate, max_depth, n_estimators))
a1, a2 = eva(xgb_model)

Train Complete, Evaluating...
0.6058999018645731 0.3773552317548047
